In [4]:

import torch
import torch.nn as nn
import numpy as np
import cv2
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import os
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
import segmentation_models_pytorch as smp
import torch.nn.functional as F
import tqdm
from PIL import Image
import random
from pytorch_lightning import  seed_everything
from sklearn.model_selection import train_test_split
import glob
import torch


7793216614622430062

In [1]:
# model segformerb4 settings
from mmseg.models import build_segmentor
from mmcv.runner.optimizer import build_optimizer

def segformer():
    model = dict(
        type='EncoderDecoder',
        backbone=dict(
            type='MixVisionTransformer',
            in_channels=3,
            embed_dims=64,
            num_stages=4,
            num_layers=[2, 2, 2, 2],
            num_heads=[1, 2, 5, 8],
            patch_sizes=[7, 3, 3, 3],
            sr_ratios=[8, 4, 2, 1],
            out_indices=(0, 1, 2, 3),
            mlp_ratio=4,
            qkv_bias=True,
            drop_rate=0.0,
            attn_drop_rate=0.0,
            drop_path_rate=0.1,
            pretrained="/mnt/sdd/nguyen.van.quan/BKAI-kaggle/pretrained/mit_b1_mmseg.pth"),
        decode_head=dict(
            type='UPerHeadV3',
            in_channels=[64, 128, 320, 512],
            in_index=[0, 1, 2, 3],
            channels=128,
            dropout_ratio=0.1,
            num_classes=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            align_corners=False,
            loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
        # model training and testing settings
        train_cfg=dict(),
        test_cfg=dict(mode='whole'))
    model = build_segmentor(model)
    model.init_weights()
    return model
    

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6
model = segformer()
import torch
model.forward_dummy(torch.rand((2, 3, 512, 512))).shape

2022-08-21 20:35:41.694202: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/mnt/sdd/nguyen.van.quan/Researchs/Polyp/mmseg/models/backbones/mit.py:365: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/mnt/sdd/nguyen.van.quan/Researchs/Polyp/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
2022-08-21 20:35:44,151 - mmcv - INFO - initialize MixVisionTransformer with init_cfg {'type': 'Pretrained', 'checkpoint': '/mnt/sdd/nguyen.van.quan/B

torch.Size([2, 512, 128, 128])


torch.Size([2, 1, 512, 512])

In [2]:
count_parameters(model)

21998875

In [1]:
from mmseg.models.decode_heads.rcfpn import RCFPN
import torch
rfp = RCFPN([64, 128, 320, 512], 128, 5)
test1 = torch.rand((2, 64, 128, 128))
test2 = torch.rand((2, 128, 64, 64))
test3 = torch.rand((2, 320, 32, 32))
test4 = torch.rand((2, 512, 16, 16))

rfp_out = rfp([test1, test2, test3, test4])


2022-08-21 20:21:20.030016: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


4
4


In [2]:

from mmseg.models.decode_heads.lib.bifpn import *

bifpn = BiFPN([64, 128, 320, 512], 128)

test1 = torch.rand((2, 64, 128, 128))
test2 = torch.rand((2, 128, 64, 64))
test3 = torch.rand((2, 320, 32, 32))
test4 = torch.rand((2, 512, 16, 16))

bifpn_out = bifpn([test1, test2, test3, test4])
for i in bifpn_out:
    print(i.shape)


torch.Size([2, 128, 128, 128])
torch.Size([2, 128, 64, 64])
torch.Size([2, 128, 32, 32])
torch.Size([2, 128, 16, 16])
torch.Size([2, 128, 8, 8])
